In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from causal_nh.utils import open_pkl_file
from causal_nh.test import test_prediction
import os
from causal_nh.train import train_nh, train_causal_nh
from causal_nh.model.NeuralHawkes import CausalNeuralHawkesMasked, CausalNeuralHawkesMaskedWeighted, CausalNeuralHawkesTrainableWeighted
import pickle
%matplotlib inline

In [70]:
data_path = "../../data"
results = "../../results"
artifacts = "../../artefacts"
if not os.path.exists(results):
    os.mkdir(results)
if not os.path.exists(artifacts):
    os.mkdir(artifacts)

datasets = [item for item in os.listdir(data_path) if os.path.isdir(f'{data_path}/{item}')]
print(datasets)
datasets = [datasets[5]]
print(datasets)

In [ ]:
new_models = [CausalNeuralHawkesMasked, CausalNeuralHawkesMaskedWeighted, CausalNeuralHawkesTrainableWeighted]
for dataset in datasets:
    dataset_name = dataset
    path = f'{data_path}/{dataset_name}/'
    print(path)
    train = open_pkl_file(path + 'train.pkl', 'train')
    dev = open_pkl_file(path + 'dev.pkl', 'dev')
    test_data = open_pkl_file(path + 'test.pkl', 'test')
    with open(path + 'A.pkl', 'rb') as f:
        A = pickle.load(f)
    A = A['A']
    W = A
    A[A==0.2] = 1

    output_folder = f'{artifacts}/{dataset_name}'
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
    
    save_path, model_path = train_nh(train, dev, f'{output_folder}/', False, lr=0.01, num_epochs=100, batch_size=50)
    
    original_time, original_type, estimated_times, estimated_types, estimated_intensities, rmse, acc = test_prediction(
                test_data = test_data,
                idx_test_series=5,
                n_samples=1000,
                start_test_idx=10,
                path_to_save=save_path,
                path_to_model=model_path,
                dataset_name=dataset_name)
    
    fname = save_path+model_path.split('/')[-1].split('.')[0]+"_metrics.txt"
    with open(fname, 'w') as f:
        f.write(f"RMSE: {rmse}\n")
        f.write(f"Accuracy: {acc}")
    
    for m in new_models:
        print(m.__name__)
        save_path, model_path = train_causal_nh(train, dev, A,W,  f'{output_folder}/', False, lr=0.01, num_epochs=100, batch_size=50, dataset_name=dataset_name, model=m)
        
        original_time, original_type, estimated_times, estimated_types, estimated_intensities, rmse, acc = test_prediction(
                test_data = test_data,
                idx_test_series=5,
                n_samples=1000,
                start_test_idx=10,
                path_to_save=save_path,
                path_to_model=model_path,
                dataset_name=dataset_name)
        fname = save_path+model_path.split('/')[-1].split('.')[0]+"_metrics.txt"
        with open(fname, 'w') as f:
            f.write(f"RMSE: {rmse}\n")
            f.write(f"Accuracy: {acc}")